In [1]:
import pandas as pd
import numpy as np
import json
import requests
import time
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects
%matplotlib inline
import seaborn as sns
from bs4 import BeautifulSoup
from functools import reduce
import re

<H1>Data Collection</H1>
The data required are:
<ul>
    <li>Sydney suburbs with postcode</li>
    The postcode is required by the <a href="ttps://www.propertyvalue.com.au">Property Value</a> site we are going to use to retrieve the property price.
    <li>Suburb distance from Sydney CBD</li>
    <li>Sydney suburbs with geocode</li>
    The geocode is required by FourSquare location data to calculate the liveable indicator of each suburb.
    <li>Liveable indicators for each suburb</li>
    <li>Population of each suburb</li>
    <li>Total crime of each suburb</li>
    <li>Property price of each suburb</li>
</ul>


<H3>1.  Sydney suburbs with postcode</H3>
As list of sydney suburbs with postcode are available at <a href="http://www.justweb.com.au/post-code/sydney-postalcodes.html">JustWeb</a>.  I have also managed to find the distance from each suburb to CBD from <a href='https://www.digitaladvocates.com.au/list-of-sydney-suburbs/'>Digital Advocates</a>.  Since distance from CBD is one of our feature selection, we can quickly merge this 2 files to get a list of sydney suburbs with postcode and distance from CBD.


First, load the sydney postcode file.

In [2]:
df_postcode = pd.read_csv("resources/sydney_postcodes.csv")
print('No of rows: {}, No of colums: {}'.format(df_postcode.shape[0], df_postcode.shape[1]))
df_postcode.head()

No of rows: 2604, No of colums: 2


,Postcode,Suburb
0,2176,Abbotsbury
1,2046,Abbotsford
2,2325,Aberdare
3,2336,Aberdeen
4,2326,Abermain


<H3>2.  Sydney suburbs with distance from CBD</H3>
Then, load the sydney suburb list with distance from CBD.

In [3]:
df_distance = pd.read_csv("resources/sydney_suburbs_distance.csv")
print('No of rows: {}, No of colums: {}'.format(df_distance.shape[0], df_distance.shape[1]))
df_distance.head()

No of rows: 951, No of colums: 3


,Suburb,Direction,Distance (km)
0,Abbotsbury,West,31.4
1,Abbotsford,West,7.4
2,Aberdare,North,115.2
3,Acacia Gardens,West,31.0
4,Adamstown,North-east,114.7


<H3>3.  Getting coordinates for the Sydney suburbs</H3>
First, install geocoder package

In [11]:
!conda install -c conda-forge geocoder --yes

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs:
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geocoder-1.38.1            |             py_0          52 KB  conda-forge
    orderedset-2.0             |           py36_0         231 KB  conda-forge
    ratelim-0.1.6              |           py36_0           5 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         288 KB

The following NEW packages will be INSTALLED:

  geocoder           conda-forge/noarch::geocoder-1.38.1-py_0
  orderedset         conda-forge/linux-64::orderedset-2.0-py36_0
  ratelim            conda-forge/linux-64::ratelim-0.1.6-py36_0



orderedset-2.0       | 231 KB    | ##################################### | 100% 
geocoder-1.38.1      | 52 KB    

After the package has been installed, import geocoder, pass in the postal code to retrieve the latitude and longtitude of each neighborhood in the dataframe.

In [13]:
import geocoder # import geocoder

def get_coordinates(suburb):
    lat_lng_coords = None
    g = geocoder.arcgis('{}, New South Wales, AUS'.format(suburb))
    lat_lng_coords = g.latlng
    
    return lat_lng_coords

for index, row in df_distance.iterrows():
    coords = get_coordinates(row['Suburb'])
    df_distance.loc[index, 'Latitude'] = coords[0]
    df_distance.loc[index, 'Longitude'] = coords[1]

df_distance.head()

,Suburb,Direction,Distance (km),Latitude,Longitude
0,Abbotsbury,West,31.4,-33.87285,150.86721
1,Abbotsford,West,7.4,-33.85041,151.12846
2,Aberdare,North,115.2,-32.84186,151.36986
3,Acacia Gardens,West,31.0,-33.73240,150.91300
4,Adamstown,North-east,114.7,-32.93434,151.72587


Let's write the records into file for further use so that we do not need to rerun the geocoder package everytime.

In [14]:
df_distance.to_csv('resources/sydney_geocode.csv', index=False)

In [4]:
df_geocode = pd.read_csv('resources/sydney_geocode.csv')
df_geocode.head()

,Suburb,Direction,Distance (km),Latitude,Longitude
0,Abbotsbury,West,31.4,-33.87285,150.86721
1,Abbotsford,West,7.4,-33.85041,151.12846
2,Aberdare,North,115.2,-32.84186,151.36986
3,Acacia Gardens,West,31.0,-33.73240,150.91300
4,Adamstown,North-east,114.7,-32.93434,151.72587


<H3>4.  Getting population by suburb</H3>

In [59]:
def getPopulation(suburbs):
    
    suburb_population_list = []
    
    for suburb in suburbs:
        
        url = "https://en.wikipedia.org/wiki/{},_New_South_Wales".format(suburb.replace(' ', '_'))
        print(url)
        
        # Getting the webpage, creating a Response object.
        headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'}
        response = requests.get(url, headers=headers)

        # Extracting the source code of the page.
        data = response.text
        
        soup = BeautifulSoup(data)
        
        # retrieve population
        th_population = soup.find('th', text='Population')
        
        population = 0
        
        if th_population is not None:
            tr = th_population.find_parent('tr')
            children = tr.findChildren("td" , recursive=False)
        
            population = children[0].get_text().split('\xa0')[0].replace(',','')
        
        # retrieve density
        th_density = soup.find('th', text=re.compile(r'Density'))
        
        density = ''
        
        if th_density is not None:
            tr = th_density.find_parent('tr')
            children = tr.findChildren("td" , recursive=False)
            span = children[0].findChildren('span', recursive=False)
            
            if len(span) > 0:
                density = round(float(span[0].get_text().replace(',','')))
            else:
                density = children[0].get_text().split('/km2')[0].replace(',','')
            
        # retrieve area
        th_area = soup.find('th', text='Area')
        
        area = ''
        
        if th_area is not None:
            tr = th_area.find_parent('tr')
            children = tr.findChildren("td" , recursive=False)
        
            area = children[0].get_text().split('km2')[0].replace(',','')
        
        suburb_population_list.append([(suburb, population, density, area)])
    
    df_suburb_population = pd.DataFrame([item for suburb_population in suburb_population_list for item in suburb_population])
    df_suburb_population.columns = ['Suburb', 
                                    'Population',
                                    'Density (/km2)',
                                    'Area (km2)']
    
    return df_suburb_population

In [ ]:
df_sydney_population = getPopulation(suburbs=df_distance['Suburb'])
df_sydney_population.head()

In [61]:
df_sydney_population.to_csv('resources/sydney_population.csv', index=False)

In [5]:
df_sydney_population = pd.read_csv('resources/sydney_population.csv')
df_sydney_population.head()

,Suburb,Population,Density (/km2),Area (km2)
0,Abbotsbury,4253,3272.0,1.3
1,Abbotsford,5373,5373.0,1
2,Aberdare,2473,750.0,3.2961
3,Acacia Gardens,3798,3798.0,1
4,Adamstown,6044,2015.0,3


<H3>Getting crime rate by suburb</H3>

In [6]:
df_crime = pd.read_csv('resources/sydney_total_crime_2016.csv')
df_crime.shape    

(4459, 2)

<H3>4.  Getting property prices by suburb</H3>


Define a function to retrieve data from PropertyValue site using BeautifulSoup package.

In [63]:
def getPropertyPrice(suburbs, postcodes):
    
    suburb_price_list = []
    
    for suburb, postcode in zip(suburbs, postcodes):
        
        url = "http://www.realestate.com.au/neighbourhoods/{}-{}-nsw".format(suburb, postcode)
        print(url)
        
        # Getting the webpage, creating a Response object.
        headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'}
        response = requests.get(url, headers=headers)

        # Extracting the source code of the page.
        data = response.text
        
        # Passing the source code to BeautifulSoup to create a BeautifulSoup object for it.
        soup = BeautifulSoup(data, 'lxml')
        
        # Extracting all the <div> tags with class=price h1 strong (this div contains house/unit price info) into a list.
        divs = soup.find_all("div", {"class": "price h1 strong"})
        
        price_info = []
        
        if len(divs) > 0:
        
            # 1st div = House Median Price
            # 2nd div = House Rent per Week
            # 3rd div = Unit Median Price
            # 4th div = Unit Rent per Week
            for i in range(0,4):
                info = divs[i].get_text()
                price_info.append(info)
        
            house_median_price = price_info[0].replace('$', '').replace(',', '')
            house_rent = price_info[1].replace('$', '').replace('PW', '')
            unit_median_price = price_info[2].replace('$', '').replace(',', '')
            unit_rent = price_info[3].replace('$', '').replace('PW', '')
        
            # get trend data
            house_price_2018 = 0
            house_price_2017 = 0
            unit_price_2018 = 0
            unit_price_2017 = 0
            house_price_change = 0
            unit_price_change = 0
            
            trend_div = soup.find('div', {'class':'slide-section median-price-subsections trend'})
            
            if trend_div is not None:
                trend_data = trend_div['data-trend']
                trend_data_json = json.loads(trend_data)
                house_trend = trend_data_json['12_months_median']['house']
                unit_trend = trend_data_json['12_months_median']['unit']
                house_price_2018 = house_trend['2018-12-31']['price'] if '2018-12-31' in house_trend else 0
                house_price_2017 = house_trend['2017-12-31']['price'] if '2017-12-31' in house_trend else 0
                unit_price_2018 = unit_trend['2018-12-31']['price'] if '2018-12-31' in unit_trend else 0
                unit_price_2017 = unit_trend['2017-12-31']['price'] if '2017-12-31' in unit_trend else 0
                house_price_change = round(((house_price_2018 - house_price_2017)/house_price_2017)*100, 2) if house_price_2017 > 0 else 0
                unit_price_change = round(((unit_price_2018 - unit_price_2017)/unit_price_2017)*100, 2) if unit_price_2017 > 0 else 0
        
            suburb_price_list.append([(suburb, postcode, house_median_price, house_rent, unit_median_price, unit_rent,
                                  house_price_2018, house_price_2017, house_price_change,
                                  unit_price_2018, unit_price_2017, unit_price_change
                                  )])
        
        time.sleep(1)
        
    df_suburb_price = pd.DataFrame([item for suburb_price in suburb_price_list for item in suburb_price])
    df_suburb_price.columns = ['Suburb', 
                  'Postcode',
                  'House Median Price', 
                  'House Rent (PW)', 
                  'Unit Median Price',
                  'Unit Rent (PW)',
                  'House Price 2018',
                  'House Price 2017',
                  'House Price Change (1 yr)',
                  'Unit Price 2018',
                  'Unit Price 2017',
                  'Unit Price Change (1 yr)']
    
    return df_suburb_price

In [ ]:
df_sydney_suburb_price = getPropertyPrice(suburbs=df_postcode['Suburb'],
                                         postcodes=df_postcode['Postcode'])
df_sydney_suburb_price.head()

In [ ]:
df_sydney_suburb_price.shape

Quicky save the dataframe into file so that we do not need to rerun this data scrapping process again.

In [20]:
df_sydney_suburb_price.to_csv('resources/sydney_suburb_price.csv', index=False)

In [53]:
df_sydney_suburb_price = pd.read_csv('resources/sydney_suburb_price.csv')
df_sydney_suburb_price.head()

,Suburb,Postcode,House Median Price,House Rent (PW),Unit Median Price,Unit Rent (PW),House Price 2018,House Price 2017,House Price Change (1 yr),Unit Price 2018,Unit Price 2017,Unit Price Change (1 yr)
0,Abbotsbury,2176,1048750,540,no data,no data,1068750,976000,9.50,0,0,0.00
1,Abbotsford,2046,2810000,820,1115000,600,2900000,2152000,34.76,1100000,1162000,-5.34
2,Aberdare,2325,366000,305,280000,280,360000,333500,7.95,277500,261250,6.22
3,Acacia Gardens,2763,820000,560,no data,no data,815000,875000,-6.86,705250,700000,0.75
4,Adamstown,2289,701000,460,485000,350,715000,710500,0.63,485000,447500,8.38


In [44]:
df_sydney_suburb_price.shape

(808, 12)

<H3>5. Analyse the features of each suburb</H3>
Let's get explore the venues around each Sydney suburb with FourSquare location data.

First, let's define our FourSquare credentials.

In [5]:
CLIENT_ID = 'KK1LKXMMYRRO0MMKH0U3GYUOWTBTT05MOF4L5ROVQTVNCNFA' # your Foursquare ID
CLIENT_SECRET = 'NNLQLSULNYS5G3FXA2NMT5AJWG5FOVISWLPNCOHDD0ONJ302' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [6]:
def extractVenueCategories(start):
    
    venue_categories = []
    
    # recursive function that collects all the ids in `acc`
    def recurse(list, data):
        
        for cat in data:
            list.append(cat['name'])
            sub_categories = cat['categories']
            
            if len(sub_categories) > 0:
                # recursive call for each subfolder
                recurse(list, sub_categories)
            
    for cat in start:
        main_categories = []
        sub_categories = cat['categories']
        
        if len(sub_categories) > 0:
            recurse(main_categories, sub_categories)
            
        venue_categories.append([(cat['name'], main_categories)])
    
    return venue_categories

def getVenueCategories():
    
    url = 'https://api.foursquare.com/v2/venues/categories?client_id={}&client_secret={}&v={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION)
    print(url)

    results = requests.get(url).json()

    df_venue_categories = pd.DataFrame(item for cat in extractVenueCategories(results['response']['categories']) for item in cat)
    df_venue_categories.columns = ['Main Category', 'Sub Category']

    category_list = []

    for index, row in df_venue_categories.iterrows():
        main_category = row['Main Category']
        sub_category_list = []
    
        for sub_category in row['Sub Category']:
            sub_category_list.append(sub_category)
    
        category_list.append([(main_category, sub) for sub in sub_category_list])

    df_venue_categories = pd.DataFrame(item for cat in category_list for item in cat)
    df_venue_categories.columns = ['Main Category', 'Sub Category']    
    
    return df_venue_categories

Retrieve venue categories from Foursquare, save it into csv for later use.

In [7]:
df_venue_categories = getVenueCategories()
df_venue_categories.head(10)

https://api.foursquare.com/v2/venues/categories?client_id=KK1LKXMMYRRO0MMKH0U3GYUOWTBTT05MOF4L5ROVQTVNCNFA&client_secret=NNLQLSULNYS5G3FXA2NMT5AJWG5FOVISWLPNCOHDD0ONJ302&v=20180605


,Main Category,Sub Category
0,Arts & Entertainment,Amphitheater
1,Arts & Entertainment,Aquarium
2,Arts & Entertainment,Arcade
3,Arts & Entertainment,Art Gallery
4,Arts & Entertainment,Bowling Alley
5,Arts & Entertainment,Casino
6,Arts & Entertainment,Circus
7,Arts & Entertainment,Comedy Club
8,Arts & Entertainment,Concert Hall
9,Arts & Entertainment,Country Dance Club


In [8]:
df_venue_categories.to_csv('resources/venue_categories.csv', index=False)

In [1]:
def getTopVenues(suburbs, latitudes, longitudes, radius=500, LIMIT=100):
    
    fixed_columns = ['Suburb', 'Latitude', 'Longitude', 'Venue', 'Category', 'Main Category'] 
    #venues =pd.DataFrame(columns = fixed_columns)
    
    venue_data = []
    
    df_main_categories = pd.read_csv('resources/venue_categories.csv')
    
    for suburb, lat, lng in zip(suburbs, latitudes, longitudes):
        print(suburb)
        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&day=any&time=any&radius={}&limit={}'.format(
                CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION, 
                lat, 
                lng, 
                radius, 
                LIMIT)
            
        results = requests.get(url).json()["response"]['groups']
        
        if len(results) > 0:
            items = results[0]['items']
            
            if len(items) > 0:
                
                for venue in items:
                    name = venue['venue']['name']
                    category = venue['venue']['categories'][0]['name']
                    main_category_row = df_main_categories.loc[df_main_categories['Sub Category'] == category, 'Main Category']
                    
                    if len(main_category_row) == 0:
                        main_category = category
                    else:
                        main_category = main_category_row.values[0]
            
                    venue_data.append([(suburb, lat, lng, name, category, main_category)])
        
    df_venue = pd.DataFrame(item for v in venue_data for item in v)
    df_venue.columns = ['Suburb', 'Latitude', 'Longitude', 'Venue', 'Category', 'Main Category']   
        
    return(df_venue)

In [ ]:
venues = getTopVenues(suburbs=df_sydney_profiles['Suburb'],
                     latitudes=df_sydney_profiles['Latitude'],
                     longitudes=df_sydney_profiles['Longitude']
                    )
venues

In [ ]:
venues = getTopVenues(suburbs=df_geocode.iloc[900:951]['Suburb'],
                     latitudes=df_geocode.iloc[900:951]['Latitude'],
                     longitudes=df_geocode.iloc[900:951]['Longitude']
                    )
venues

In [7]:
venues.to_csv("resources/sydney_suburb_venues_new.csv", index=False)

In [83]:
with open('resources/sydney_suburb_venues.csv', 'a') as f:
             venues.to_csv(f, header=False, index=False)

In [9]:
df_venues = pd.read_csv("resources/sydney_suburb_venues_new.csv")
df_venues.head()

,Suburb,Latitude,Longitude,Venue,Category,Main Category
0,Abbotsbury,-33.87285,150.86721,Stockdale reserve,Park,Outdoors & Recreation
1,Abbotsbury,-33.87285,150.86721,Abbotsbury Shops,Convenience Store,Shop & Service
2,Abbotsbury,-33.87285,150.86721,817 bus stop,Bus Station,Travel & Transport
3,Abbotsford,-33.85041,151.12846,The Cove Dining Co,Café,Food
4,Abbotsford,-33.85041,151.12846,The Cove Dining Co,Café,Food


In [10]:
df_venues = df_venues[['Suburb', 'Main Category']]
df_venues.head()

,Suburb,Main Category
0,Abbotsbury,Outdoors & Recreation
1,Abbotsbury,Shop & Service
2,Abbotsbury,Travel & Transport
3,Abbotsford,Food
4,Abbotsford,Food


In [11]:
# one hot encoding
venues_onehot = pd.get_dummies(df_venues[['Main Category']], prefix="", prefix_sep="")

# add suburb column back to dataframe
venues_onehot['Suburb'] = df_venues['Suburb'] 

# move suburb column to the first column
fixed_columns = [venues_onehot.columns[-1]] + list(venues_onehot.columns[:-1])
venues_onehot = venues_onehot[fixed_columns]

venues_onehot.head(20)

,Suburb,Arts & Entertainment,College & University,Food,Nightlife Spot,Outdoors & Recreation,Professional & Other Places,Shop & Service,Travel & Transport
0,Abbotsbury,0,0,0,0,1,0,0,0
1,Abbotsbury,0,0,0,0,0,0,1,0
2,Abbotsbury,0,0,0,0,0,0,0,1
3,Abbotsford,0,0,1,0,0,0,0,0
4,Abbotsford,0,0,1,0,0,0,0,0
5,Abbotsford,0,0,0,0,1,0,0,0
6,Abbotsford,0,0,1,0,0,0,0,0
7,Abbotsford,0,0,0,0,0,0,1,0
8,Abbotsford,0,0,1,0,0,0,0,0
9,Abbotsford,0,0,1,0,0,0,0,0


In [12]:
venues_grouped = venues_onehot.groupby('Suburb').sum().reset_index()
venues_grouped

,Suburb,Arts & Entertainment,College & University,Food,Nightlife Spot,Outdoors & Recreation,Professional & Other Places,Shop & Service,Travel & Transport
0,Abbotsbury,0,0,0,0,1,0,1,1
1,Abbotsford,0,0,5,0,1,0,1,0
2,Aberdare,0,0,1,1,0,0,1,1
3,Acacia Gardens,0,0,0,0,1,0,0,0
4,Adamstown,0,0,3,2,1,0,1,0
5,Adamstown Heights,0,0,0,0,0,0,1,0
6,Airds,0,0,0,1,0,0,2,0
7,Alexandria,0,0,15,1,1,0,3,0
8,Alfords Point,0,0,1,0,1,0,3,0
9,Allambie Heights,0,0,2,0,0,0,2,0


In [ ]:
venues_grouped.to_csv('resources/syndey_suburb_venues_count.csv', index=False)

In [26]:
df_venues_grouped = pd.read_csv('resources/syndey_suburb_venues_count.csv')
df_venues_grouped

,Suburb,Arts & Entertainment,College & University,Food,Nightlife Spot,Outdoors & Recreation,Professional & Other Places,Shop & Service,Travel & Transport
0,Abbotsbury,0,0,0,0,1,0,2,1
1,Abbotsford,0,0,5,0,1,0,1,0
2,Aberdare,0,0,1,1,0,0,1,1
3,Acacia Gardens,0,0,1,0,1,0,0,0
4,Adamstown,0,0,3,2,0,0,1,0
5,Adamstown Heights,0,0,0,0,1,0,1,0
6,Airds,0,0,0,1,1,0,1,0
7,Alexandria,0,0,15,1,1,0,5,0
8,Alfords Point,0,0,1,0,1,0,3,0
9,Alison,0,0,2,1,0,0,1,0


Now, merge all the datasource into one dataset.

In [54]:
data_frames = [df_postcode, df_distance, df_geocode, df_sydney_population, df_crime, df_sydney_suburb_price, df_venues_grouped]
df_sydney_profiles = reduce(lambda  left,right: pd.merge(left,right,on=['Suburb'],
                                            how='inner'), data_frames)
df_sydney_profiles.head()

,Postcode_x,Suburb,Direction_x,Distance (km)_x,Direction_y,Distance (km)_y,Latitude,Longitude,Population,Density (/km2),...,Unit Price 2017,Unit Price Change (1 yr),Arts & Entertainment,College & University,Food,Nightlife Spot,Outdoors & Recreation,Professional & Other Places,Shop & Service,Travel & Transport
0,2176,Abbotsbury,West,31.4,West,31.4,-33.87285,150.86721,4253,3272.0,...,0,0.00,0,0,0,0,1,0,2,1
1,2046,Abbotsford,West,7.4,West,7.4,-33.85041,151.12846,5373,5373.0,...,1162000,-5.34,0,0,5,0,1,0,1,0
2,2325,Aberdare,North,115.2,North,115.2,-32.84186,151.36986,2473,750.0,...,261250,6.22,0,0,1,1,0,0,1,1
3,2763,Acacia Gardens,West,31.0,West,31.0,-33.73240,150.91300,3798,3798.0,...,700000,0.75,0,0,1,0,1,0,0,0
4,2289,Adamstown,North-east,114.7,North-east,114.7,-32.93434,151.72587,6044,2015.0,...,447500,8.38,0,0,3,2,0,0,1,0


In [48]:
df_sydney_profiles.shape

(769, 31)

In [56]:
df_sydney_profiles.columns

Index(['Postcode_x', 'Suburb', 'Direction_x', 'Distance (km)_x', 'Direction_y',
       'Distance (km)_y', 'Latitude', 'Longitude', 'Population',
       'Density (/km2)', 'Area (km2)', 'Total', 'Postcode_y',
       'House Median Price', 'House Rent (PW)', 'Unit Median Price',
       'Unit Rent (PW)', 'House Price 2018', 'House Price 2017',
       'House Price Change (1 yr)', 'Unit Price 2018', 'Unit Price 2017',
       'Unit Price Change (1 yr)', 'Arts & Entertainment',
       'College & University', 'Food', 'Nightlife Spot',
       'Outdoors & Recreation', 'Professional & Other Places',
       'Shop & Service', 'Travel & Transport'],
      dtype='object')

In [57]:
df_sydney_profiles = df_sydney_profiles[['Postcode_x', 'Suburb', 'Direction_x', 
       'Distance (km)_x', 'Latitude', 'Longitude', 'Population', 
       'Density (/km2)', 'Area (km2)', 'Total',
       'House Median Price', 'House Rent (PW)', 'Unit Median Price',
       'Unit Rent (PW)', 'House Price 2018', 'House Price 2017',
       'House Price Change (1 yr)', 'Unit Price 2018', 'Unit Price 2017',
       'Unit Price Change (1 yr)', 'Arts & Entertainment',
       'College & University', 'Food', 'Nightlife Spot',
       'Outdoors & Recreation', 'Professional & Other Places',
       'Shop & Service', 'Travel & Transport']]
df_sydney_profiles.head()

,Postcode_x,Suburb,Direction_x,Distance (km)_x,Latitude,Longitude,Population,Density (/km2),Area (km2),Total,...,Unit Price 2017,Unit Price Change (1 yr),Arts & Entertainment,College & University,Food,Nightlife Spot,Outdoors & Recreation,Professional & Other Places,Shop & Service,Travel & Transport
0,2176,Abbotsbury,West,31.4,-33.87285,150.86721,4253,3272.0,1.3,121,...,0,0.00,0,0,0,0,1,0,2,1
1,2046,Abbotsford,West,7.4,-33.85041,151.12846,5373,5373.0,1,158,...,1162000,-5.34,0,0,5,0,1,0,1,0
2,2325,Aberdare,North,115.2,-32.84186,151.36986,2473,750.0,3.2961,271,...,261250,6.22,0,0,1,1,0,0,1,1
3,2763,Acacia Gardens,West,31.0,-33.73240,150.91300,3798,3798.0,1,60,...,700000,0.75,0,0,1,0,1,0,0,0
4,2289,Adamstown,North-east,114.7,-32.93434,151.72587,6044,2015.0,3,656,...,447500,8.38,0,0,3,2,0,0,1,0


In [58]:
df_sydney_profiles.rename(columns={'Postcode_x':'Postcode',
                          'Direction_x':'Direction',
                          'Distance (km)_x':'Distance (km)',
                          'Total':'Total Crime'}, 
                            inplace=True)
df_sydney_profiles.columns

Index(['Postcode', 'Suburb', 'Direction', 'Distance (km)', 'Latitude',
       'Longitude', 'Population', 'Density (/km2)', 'Area (km2)',
       'Total Crime', 'House Median Price', 'House Rent (PW)',
       'Unit Median Price', 'Unit Rent (PW)', 'House Price 2018',
       'House Price 2017', 'House Price Change (1 yr)', 'Unit Price 2018',
       'Unit Price 2017', 'Unit Price Change (1 yr)', 'Arts & Entertainment',
       'College & University', 'Food', 'Nightlife Spot',
       'Outdoors & Recreation', 'Professional & Other Places',
       'Shop & Service', 'Travel & Transport'],
      dtype='object')

In [59]:
postcode_duplicate_count = df_sydney_profiles.groupby(['Suburb'])['Postcode'].count()
postcode_duplicate_count[postcode_duplicate_count > 1]

Suburb
Alexandria          2
Artarmon            2
Ashfield            2
Auburn              2
Bankstown           2
Baulkham Hills      2
Beaconsfield        2
Bondi Junction      2
Botany              2
Burwood             2
Camperdown          2
Caringbah           2
Castle Hill         2
Chatswood           2
Crows Nest          2
Darlinghurst        2
Double Bay          2
Drummoyne           2
Epping              2
Ermington           2
Fairfield           2
Frenchs Forest      3
Gladesville         2
Haymarket           2
Hornsby             2
Ingleburn           2
Kensington          2
Kingsgrove          2
Kogarah             2
Lane Cove           2
Lidcombe            2
Liverpool           2
Manly               2
Marrickville        2
Mascot              2
Milsons Point       2
Miranda             2
Mona Vale           2
Moorebank           2
Mount Lewis         2
North Parramatta    2
North Sydney        2
Northbridge         2
Parramatta          3
Pennant Hills       2
Pen

In [60]:
df_sydney_profiles = df_sydney_profiles.drop_duplicates(subset=['Suburb'], keep='first')
df_sydney_profiles.shape

(709, 28)

In [61]:
df_sydney_profiles.to_csv('resources/sydney_suburb_profiles.csv', index=False)